In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message system allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import (
    InMemoryConversationMemory,
    ToolConfig,
    ToolParameter,
    ToolProperty,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool definition

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [6]:
prompt = "Turn this place into a comfortable party mood!"

### OpenAI Chat Completion

ToolConfig object is internally converted to client tool configs

In [7]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [8]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party is ready to rock! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are dimmed for the perfect atmosphere. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=134, completion_tokens=37, total_tokens=171), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_85oWCBUl3U86loELycmpDNGd', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_piPDMWyNeJVKT6r2oI9RmhAL', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_xFdpvOdIhlmkKsziyZSHZ5Qh', 'function': {'arguments': '{"brightness": 0.3}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': '

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party is ready to rock! The disco ball is spinning, energetic music is playing at 120 BPM, and the lights are dimmed for the perfect atmosphere. Enjoy the party!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 134,
  'completion_tokens': 37},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_85oWCBUl3U86loELycmpDNGd',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_piPDMWyNeJVKT6r2oI9RmhAL',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_xFdpvOdIhlmkKsziyZSHZ5Qh',
     'function': {'arguments': '

Asynchronous generation

In [10]:
response = await chat_openai.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The place is now set for a comfortable party mood! The disco ball is spinning, energetic music is playing at a lively 120 bpm, and the lights are dimmed to create the perfect atmosphere. Enjoy the party!')], name=None), usage=Usage(prompt_tokens=134, completion_tokens=44, total_tokens=178), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': 'User'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_zGHKavv1gwDLH0c0siJsTY4k', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_bm7wkchdfHT5gNPdsSVVm1uR', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_RKWt5QFDRM74PWp6dXuijpzK', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_light

multiple generation

In [11]:
response = chat_openai.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The place is now in a comfortable party mood! The lights are dimmed, energetic music is playing at a lively 120 BPM, and the disco ball is spinning. Enjoy the party!'},
  {'text': 'The place is now set for a comfortable party mood! The lights are dimmed to create the right ambiance, energetic music is playing at 120 bpm, and the disco ball is spinning its magic. Enjoy the party!'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chat_openai.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've started some energetic music to set the party mood! Let me know if there's anything else you need to make this place feel more lively!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 64,
  'completion_tokens': 28},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_2fegUgC5CmzxZZTqbvWqEtzm',
     'function': {'arguments': '{"energetic":true,"loud":true,"bpm":"120"}',
      'name': 'start_music'},
     'type': 'function'}]},
  {'role': 'tool',
   'tool_call_id': 'call_2fegUgC5CmzxZZTqbvWqEtzm',
   'name': 'start_music',
   'content': "Starting music! energetic=True loud=True, bpm='120'"}]}

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chat_openai.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=171, completion_tokens=75, total_tokens=246), results=[Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_UkDvmOyWNYu4RZyGGiO73ogX', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_0UC2UXaVGtMAR69iGOoN0h82', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights'), Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_pMIHur0uOhZBMMEEZmQN50n4', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball')], calls=Message(role='function_call', content=[ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_UkDvmOyWNYu4RZyGGiO73ogX'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_0UC

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 171,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_UkDvmOyWNYu4RZyGGiO73ogX',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_0UC2UXaVGtMAR69iGOoN0h82',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'},
  {'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_pMIHur0uOhZBMMEEZmQN50n4',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_UkDv

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_UkDvmOyWNYu4RZyGGiO73ogX', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_0UC2UXaVGtMAR69iGOoN0h82', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights'),
 Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_pMIHur0uOhZBMMEEZmQN50n4', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_UkDvmOyWNYu4RZyGGiO73ogX'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_0UC2UXaVGtMAR69iGOoN0h82'), ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_pMIHur0uOhZBMMEEZmQN50n4')], name=None)

# Pydantic tools

You can define tools as pydantic models. To use pydantic model as tool, make sure the following points: 

- Write description of top level docstring field in the class. This is used for tool description.
- Define each argument fileds with description
- Implement run() method.

Tool definition using `ToolProperty`, `ToolParameter` and `ToolConfig` is almost equivalent to the following way. 

In [17]:
from enum import Enum
from typing import Literal

from pydantic import BaseModel, Field


class PowerDiscoBall(BaseModel):
    """Powers the spinning disco ball."""

    power: bool = Field(description="Boolean to spin disco ball.")

    def run(self) -> str:
        return f"Disco ball is {'spinning!' if self.power else 'stopped.'}"


class DimLights(BaseModel):
    """Dim the lights."""

    brightness: float = Field(description="The brightness of the lights, 0.0 is off, 1.0 is full.")

    def run(self) -> str:
        return f"Lights are now set to {self.brightness}"


class BPM(Enum):
    """The beats per minute of the music."""

    SLOW = "60"
    MEDIUM = "120"
    FAST = "180"


class StartMusic(BaseModel):
    """Play some music matching the specified parameters."""

    energetic: bool = Field(description="Whether the music is energetic or not.")
    loud: bool = Field(description="Whether the music is loud or not.")
    bpm: BPM = Field(description="The beats per minute of the music.")
    # bpm: Literal["60", "120", "180"] = Field(..., description="The beats per minute of the music.") # This also works

    def run(self) -> str:
        return f"Starting music! {self.energetic=} {self.loud=}, {self.bpm=}"

In [18]:
# Convert to ToolConfig

tools = [PowerDiscoBall, StartMusic, DimLights]
tool_configs_pydantic = [ToolConfig.from_pydantic(tool) for tool in tools]

In [19]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_configs=tool_configs_pydantic,
)

In [20]:
response = chat_openai.run(prompt)

# Response is CompletionResults object
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! The disco ball is spinning, the energetic music is playing loudly at a lively 120 BPM, and the lights have been dimmed to create the perfect ambiance. Enjoy the celebration! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 150,
  'completion_tokens': 49},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_nfa2UmaXduldrBWj8quiN9wa',
     'function': {'arguments': '{"power": true}', 'name': 'PowerDiscoBall'},
     'type': 'function'},
    {'id': 'call_dZzToMx7DKVNO4ZPUS6CqvDG',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'StartMusic'},
     'type': 'function'},
    {'id': 'call_EQwJrPbzb1c3AvvaK8w5dLu

### Array property for OpenAI API

Function Calling with array property for OpenAI Chat Completion.

In [21]:
def get_ingredients(ingredients: list[str]) -> str:
    """Prepare ingredients to make a meal."""
    return f"Preparing {ingredients}."


tool_configs_array = [
    ToolConfig(
        name="get_ingredients",
        description="Prepare ingredients to make a meal.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="ingredients",
                    type="array",
                    description="List of ingredients.",
                    items={"type": "string"},
                ),
            ],
            required=["ingredients"],
        ),
    ),
]

In [22]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-08-06",
    tools=[get_ingredients],
    tool_configs=tool_configs_array,
)

In [23]:
response = chat_openai.run(
    prompt="What are the ingredients to make omelet rice?",
    tool_choice="get_ingredients",
)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "To make omelet rice, you'll need the following ingredients:\n\n- Rice\n- Eggs\n- Onion\n- Carrot\n- Green peas\n- Chicken (or another protein of your choice)\n- Soy sauce\n- Ketchup\n- Butter\n- Salt\n- Pepper\n- Vegetable oil\n\nThese ingredients will help you create a delicious and hearty omelet rice dish. Enjoy cooking!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-08-06',
  'prompt_tokens': 125,
  'completion_tokens': 78},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'What are the ingredients to make omelet rice?'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_pDl8lSQ6ASRhm92GwriS4HY3',
     'function': {'arguments': '{"ingredients":["rice","eggs","onion","carrot","green peas","chicken","soy sauce","ketchup","butter","salt","pepper","vegetable oil"]}',
      'name': 'get_ingredients'},
  

### Azure OpenAI Chat Completion 

In [24]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [25]:
response = chat_openai_azure.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The lights are dimmed to a cozy level, the disco ball is spinning, and the energetic music is pumping at 120 BPM. Let's get this party started!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 135,
  'completion_tokens': 33},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_3kkXzWYUSKFQPAayGEoEHY89',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_5ZgpdhzOtiLwq6F7yHHoH5va',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_DNigWCiQnPtmDKYWR3l1JnPl',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      '

In [26]:
response = await chat_openai_azure.arun(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The lights are dimmed to a cozy level, the energetic music is pumping at 120 BPM, and the disco ball is spinning! The party mood is set. Enjoy!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 135,
  'completion_tokens': 35},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_u7s2Rrv9U0rjWMfawW0kWDCx',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'},
    {'id': 'call_7W0ZZ7SJyD9cTH6GpA9jWfZY',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7bee5cr9ijXuQKZDiCQZWSru',
     'function': {'arguments': '{"power": true}', 'name'

In [27]:
response = chat_openai_azure.run(prompt, tool_choice="start_music")

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Let's get this party started! 🎉\n\n- **Lighting:** Dim the lights and set some colorful mood lighting or fairy lights.\n- **Decorations:** Add some fun decorations like balloons, streamers, and banners.\n- **Seating:** Arrange comfortable seating areas where people can relax and chat.\n- **Food & Drinks:** Set up a snack table with finger foods and a variety of drinks.\n- **Games & Activities:** Prepare some party games or a dance floor for dancing.\n\nAnything specific you need help with next?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 64,
  'completion_tokens': 105},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_amJWypa3VDqeRTRy3tviN222',
     'function': {'arguments'

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for OpenAI Chat Completion.

In [29]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [30]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone, get ready to party! I've dimmed the lights, cranked up some energetic tunes, and even got the disco ball spinning. Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 40},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
           

Asynchronous generation

In [31]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've started the music, dimmed the lights, and the disco ball is going! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 29},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
  

Gemini allows you to call specific tool as well as OpenAI Chat Completion.

In [32]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  🎶 \n\nLet's get this party started! 🥳 \n\nWhat else can I do to make this place more festive?  🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 68,
  'completion_tokens': 38},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "

Also pydantic tools is acceptable.

In [33]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [34]:
response = gemini.run(prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 149,
  'completion_tokens': 29},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "PowerDiscoBall"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "StartMusic"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
 

### Gemini on VertexAI

In [35]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [36]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm getting this party started! \n\nI've turned on some upbeat music with a tempo of 120 bpm. The lights are dimmed to a cozy level, creating a nice atmosphere. And to add a touch of fun, I've turned on the disco ball! \n\nDo you need anything else to get the party going?  Maybe some snacks or drinks? I can help with that too! \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 73,
  'completion_tokens': 90},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value:

multiple generation

In [37]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Okay, I'm getting the party started! I've turned on some upbeat music with a steady beat, dimmed the lights a bit for a cozy ambiance, and even got the disco ball spinning! \U0001faa9🎶  \n\nIs there anything else you'd like me to do to make this party even more awesome?  Let me know! 🎉 \n"},
  {'text': "Okay, I'm getting the party started! I've turned on some upbeat music with a steady beat, dimmed the lights a bit for a cozy ambiance, and even got the disco ball spinning! \U0001faa9🎶  \n\nIs there anything else you'd like me to do to make this party even more awesome?  Let me know! 🎉 \n"}],
 'name': None}

Pydantic tool use

In [38]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [39]:
response = gemini_vertexai.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, let's get this party started! \n\nI've turned on the disco ball, so the room is now sparkling with colorful light. I've also started some upbeat music with a tempo of 120 bpm, keeping it energetic but not too loud.  The lights have been dimmed to a comfortable level, creating a warm and inviting atmosphere. \n\nNow, what else can I do to make this place feel like a party? Tell me:\n\n* **What kind of party is it?**  Is it a casual get-together, a birthday bash, a themed party, or something else? \n* **What kind of music do you want?**  Do you have any specific requests?\n* **What kind of decorations would you like?**  Balloons, streamers, banners, or something else? \n* **Any other ideas you have?**\n\nLet's make this the best party ever! \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 77,
  'completion_tokens': 198},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into

### Anthropic Claude

tool_configs can be shared with Claude.

In [40]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [41]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Turned on the disco ball: This will add a fun, party atmosphere with its spinning lights.\n\n2. Started some music: I've chosen energetic music to keep the party spirit up, but not too loud so it's still comfortable for conversation. The tempo is set to a moderate 120 beats per minute, which is good for a relaxed yet upbeat mood.\n\n3. Dimmed the lights: I've set the brightness to 50% (0.5), which should create a cozy, intimate atmosphere without being too dark.\n\nThese settings should create a comfortable party mood that's lively enough for fun but still allows for easy conversation and relaxation. The combination of the spinning disco ball, upbeat yet not overpowering music, and dimmed lights should set just the right tone for your party.\n\nIs there anything else you'd like to adjust to make the mood even more to your liking?"}],
  'name': None

Asynchronous generation

In [42]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and inviting atmosphere without being too dark.\n2. Started some music that is energetic but not too loud, with a moderate tempo of 120 beats per minute. This should provide a good background for conversation and light dancing if desired.\n3. Turned on the disco ball to add a fun, festive element to the room without being overwhelming.\n\nThese settings should create a comfortable party mood that's perfect for socializing and enjoying the atmosphere. The dimmed lights and spinning disco ball will create an interesting visual effect, while the music adds energy without being too intense.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable or suited to your preferences?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620

Pydantic tool use

In [43]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [44]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up the perfect comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4). This creates a cozy, relaxed atmosphere without making it too dark.\n2. Started playing music that is energetic but not too loud, with a moderate tempo of 120 beats per minute. This should provide a nice background for conversation and light dancing if desired.\n3. Powered on the disco ball, which adds a fun, festive element to the room without being overwhelming.\n\nThese settings should create a comfortable party atmosphere that's perfect for socializing and enjoying the evening. The combination of soft lighting, upbeat but not overpowering music, and the subtle sparkle from the disco ball should set just the right mood.\n\nIs there anything else you'd like me to adjust to make the party mood even more comfortable?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'p

### Claude on Amazon Bedrock

In [45]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [46]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.4, which is low enough to create a cozy atmosphere but still allows people to see comfortably.\n\n2. Started the music: I've chosen music that is energetic to keep the party mood, but not too loud so people can still converse comfortably. The beats per minute (BPM) is set to 120, which is a moderate tempo that's good for a relaxed party vibe.\n\n3. Activated the disco ball: To add a touch of fun and create some interesting light patterns, I've turned on the disco ball.\n\nThese settings should create a comfortable party atmosphere that's lively but not overwhelming. The dimmed lights and spinning disco ball will create a fun, relaxed ambiance, while the energetic but not-too-loud music at a moderate tempo will provide a nice backdrop for socializing.\n\nIs there anything else you'd like me to adjust t

In [47]:
repsonse = await claude.arun(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.4, which is low enough to create a cozy atmosphere but still allows people to see comfortably.\n\n2. Started the music: I've chosen music that is energetic to keep the party mood, but not too loud so people can still converse comfortably. The beats per minute (BPM) is set to 120, which is a moderate tempo that's good for a relaxed party vibe.\n\n3. Activated the disco ball: To add a touch of fun and create some interesting light patterns, I've turned on the disco ball.\n\nThese settings should create a comfortable party atmosphere that's lively but not overwhelming. The dimmed lights and spinning disco ball will create a fun, relaxed ambiance, while the energetic but not-too-loud music at a moderate tempo will provide a nice backdrop for socializing.\n\nIs there anything else you'd like me to adjust t

Pydantic tool use

In [48]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[PowerDiscoBall, StartMusic, DimLights],
    tool_configs=tool_configs_pydantic,
)

In [49]:
response = claude.run(prompt=prompt)
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've made the following adjustments to create a comfortable party mood:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a warm and cozy atmosphere without being too dark.\n2. Started playing music with a moderate tempo (120 BPM). The music is energetic to keep the party vibe going, but not too loud so people can still comfortably chat.\n3. Turned on the disco ball to add a fun, festive element to the space.\n\nThese settings should create a comfortable party atmosphere that's lively enough for fun but still allows for conversation and relaxation. The dimmed lights and spinning disco ball will create an interesting visual effect, while the music adds energy without overpowering the room.\n\nIs there anything else you'd like me to adjust to perfect the party mood? For example, we could change the music tempo, adjust the lighting further, or turn off the disco ball if it's too much. Just let me know!"}],

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [50]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None, type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None, type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lig

Then these client tool config objects is transformed to client tools by format() method.

In [51]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'required': ['power'],
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}}}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [52]:
from langrila.gemini.genai import GeminiToolConfig

In [53]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None, items=None)], required=['power']), strict=None),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None, items=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None, items=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'], items=None)], required=['energetic', 'loud', 'bpm']), strict=None),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParame

In [54]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For OpenAI Chat Completion

In [55]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [56]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! The disco ball is spinning, the music is playing energetically at 120 BPM, and the lights are dimmed to create the perfect atmosphere. Let the celebration begin! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 134,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_bPGkiTxGm7g5iTpV5XrhtGFu',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_yA4AXSZ3VVeA7q0bGN2B1T6k',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_wvXyBNpZR7VO4FYO59gw1c9L',
     

Next turn

In [57]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The music's BPM has been increased to 180! Get ready to dance and have an even more energetic time! 🎶💥"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 246,
  'completion_tokens': 27},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_bPGkiTxGm7g5iTpV5XrhtGFu',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_yA4AXSZ3VVeA7q0bGN2B1T6k',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_wvXyBNpZR7VO4FYO59gw1c9L',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 

### For Gemini

In [58]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [59]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everyone! Time to get this party started! The disco ball is on, the music is pumping, and the lights are low. Let's go! 🎉 🎶 🕺 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 38},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          

Tool calling is not needed additionally if the thing is already known.

In [60]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current music bpm is 120. 🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 193,
  'completion_tokens': 12},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "dim_lights"
      args {
     

## Multi-turn conversation using tools with multiple model

The universal message and tool system allows us to chat multiple model across client.

In [62]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [63]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is all set! The disco ball is spinning, energetic music is playing at a lively 120 BPM, and the lights are dimmed to create the perfect atmosphere. Let the fun begin! 🎉✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 134,
  'completion_tokens': 44},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_FC5jRFdLgcEdqpLtzi5ccRYg',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_QxykFPPxMzMvxEhJLl6WHTJt',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7hxq6DIRa7OQZxQyVLBEMJjj',
     'fu

Claude can continue conversation without any additional process.

In [64]:
prompt = "Can you make bpm up more?"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've increased the BPM to 180, which is the highest available option. The music is now playing at a much faster tempo, perfect for an energetic dance party. The beat is quicker, and it should really get people moving on the dance floor!\n\nIs there anything else you'd like to adjust to enhance the party mood further?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1113,
  'completion_tokens': 78},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_FC5jRFdLgcEdqpLtzi5ccRYg',
     'type': 'tool_use',
     'input': {'power': True},
     'name': 'power_disco_ball'},
    {'id': 'toolu_QxykFPPxMzMvxEhJLl6WHTJt',
     'type': 'tool_use',
     'input': {'energetic': True, 'loud': True, 'bpm': '120'},
     'name': 'start_music'},
    {'id': 'toolu_7hxq6DIRa7

Go back to OpenAI Chat Completion

In [65]:
prompt = "Please turn music more relax."

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I've switched to more relaxing music with a BPM of 60. The atmosphere should now be calm and soothing, perfect for winding down or enjoying some chill conversation. Let me know if you need any further adjustments!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 377,
  'completion_tokens': 42},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_FC5jRFdLgcEdqpLtzi5ccRYg',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_QxykFPPxMzMvxEhJLl6WHTJt',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_7hxq6D

Gemini can follow as well

In [66]:
prompt = "Nice. So please go back to the initial bpm."

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright! I've switched the music back to the initial 120 BPM. It's not as relaxed as 60 BPM, but it's not overly energetic either, offering a good balance for a comfortable party atmosphere. 🎶 \n\nFeel free to ask for any other adjustments – I'm here to help you set the perfect mood! 😊 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 562,
  'completion_tokens': 75},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value

In [67]:
prompt = "OK, then turn brightness to 0.7."

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've increased the brightness of the lights to 0.7. This setting should provide a bit more illumination than before, creating a livelier atmosphere while still maintaining a party vibe. The room should now be noticeably brighter but not overly lit.\n\nTo recap the current party setup:\n1. The disco ball is spinning\n2. The music is playing at 120 BPM, energetic and loud\n3. The lights are now set at 0.7 brightness\n\nIs there anything else you'd like to adjust or any other changes you want to make to the party atmosphere?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 1707,
  'completion_tokens': 134},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a comfortable party mood!',
     'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'id': 'toolu_FC5jRFdLgcEdqpLtzi5ccRYg',
     'type': 'tool_use',
     'input': {'power': True},
     'name': '

In [68]:
prompt = "Finally stop the disco ball."

response = gemini.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "No problem, I've stopped the disco ball. It's no longer spinning. \n\nEnjoy the rest of the party! Let me know if there's anything else I can do to help fine-tune the atmosphere. 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 885,
  'completion_tokens': 48},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          valu

If you input non-relavant topic, any tool is not called.

In [69]:
prompt = "What is the weather today?"

response = chat_openai.run(prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "I'm unable to provide real-time weather information or updates since my training only goes up to October 2023 and I don't have access to live data. However, you can check a weather website or app for the most accurate and current weather conditions in your area. If you have any other questions or need assistance, feel free to ask!"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 840,
  'completion_tokens': 67},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': 'User'},
  {'content': None,
   'refusal': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_FC5jRFdLgcEdqpLtzi5ccRYg',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_QxykFPPxMzMvxEhJLl6WHTJt',
     'function': {'arguments': '{"energeti

Conversation is recorded with serialized universal message format.

In [70]:
shared_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_FC5jRFdLgcEdqpLtzi5ccRYg'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_QxykFPPxMzMvxEhJLl6WHTJt'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_7hxq6DIRa7OQZxQyVLBEMJjj'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_FC5jRFdLgcEdqpLtzi5ccRYg',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_QxykFPPxMzMvxEhJLl6WHTJt',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'},
   {'output': 'Lights are now set to 0.5',
    'call_id': 'call_7hxq6DI

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [71]:
from langrila import InMemoryConversationMemory

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [72]:
prompt = "Turn this place into a comfortable party mood!"

response = chat_openai.run(prompt, tool_choice="auto")

In [73]:
prompt = "Can you make bpm up more?"

response = chat_openai.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [74]:
print(response.message.content[0].text)

The music BPM has been increased to 180! Get ready to dance! 🎶💃🕺


In [75]:
response.usage

Usage(prompt_tokens=212, completion_tokens=22, total_tokens=234)

In [76]:
response.prompt

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_oBDXjJKvOAMcwTPBAYVDDGdY',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'},
   {'id': 'call_zIEXOL3BZiecD9yidM08GYTC',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_aR9ce1rY5DqmzxR9NBry62QP',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_oBDXjJKvOAMcwTPBAYVDDGdY',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 'tool',
  'tool_call_id': 'call_zIEXOL3BZiecD9yidM08GYTC',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_aR9ce1rY5DqmzxR9NBry62QP',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm=

In [77]:
# whole history
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_oBDXjJKvOAMcwTPBAYVDDGdY'},
   {'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_zIEXOL3BZiecD9yidM08GYTC'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_aR9ce1rY5DqmzxR9NBry62QP'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Lights are now set to 0.5',
    'call_id': 'call_oBDXjJKvOAMcwTPBAYVDDGdY',
    'args': '{"brightness": 0.5}',
    'funcname': 'dim_lights'},
   {'output': 'Disco ball is spinning!',
    'call_id': 'call_zIEXOL3BZiecD9yidM08GYTC',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'},
   {'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_aR9ce1rY5DqmzxR9NBry62QP',
    'args